### 딥러닝 모델이 예측값 구하는 방식

- 순전파(Forward propagation) : 입력 값을 바탕으로 출력 값을 계산하는 과정

- 활성화 함수는 비선형 함수(sigmoid, relu, softmax, ...)
- 순전파를 사용하면 예측값과 실제값 간의 오차값을 구하여 loss function을 구할 수 있음
- 최적화 방법 => Gradient descent : 가중치를 loss function 값이 작아지게 업데이트
- Gradient 값은 각 가중치 마다 정하지며, 역전파를 통하여 구할 수 있음
- 역전파 : Forward propagation의 반대 방향으로 이뤄지는 과정

### 딥러닝 모델의 학습 순서

1. 학습용 feature 데이터를 입력하여 예측값 구하기(순전파)
2. 예측값과 실제값 사이의 오차 구하기(Loss 계산)
3. Loss를 줄일 수 있는 가중치 업데이트 하기(역전파)
4. 1-3번 반복으로 Loss를 최소로 하는 가중치 얻기


- Epoch : 한 번의 에폭은 전체 데이터 셋에 대해 한 번 학습을 완료한 상태
- Batch : 나눠진 데이터 셋(보통 mini-batch라고 표현)
- iteration : epoch을 나누어서 실행하는 횟수

ex) 총 데이터가 천개, batch size가 100이면
- 1 iteration = 100개 데이터에 대해 학습
- 1 epoch = 10 iteration

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # warning 메시지 제거

In [6]:
df = pd.read_csv("data/Advertising.csv")
df

,ID,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9
...,...,...,...,...,...
195,196,38.2,3.7,13.8,7.6
196,197,94.2,4.9,8.1,9.7
197,198,177.0,9.3,6.4,12.8
198,199,283.6,42.0,66.2,25.5


In [7]:
df = df.drop(columns = ['ID'])

x = df.drop(columns = ['Sales'])
y = df['Sales']

train_x, test_x, train_y, test_y = train_test_split(x,y, test_size = 0.3)

In [9]:
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)) # 학습용 데이터를 tf.data.Dataset 형태로 변환
train_ds = train_ds.shuffle(len(train_x)).batch(batch_size=5) 

In [10]:
[(train_features_batch, label_batch)] = train_ds.take(1)

In [14]:
print('FB, TV, Newspaper batch 데이터:\n',train_features_batch)
print('Sales batch 데이터:',label_batch)

FB, TV, Newspaper batch 데이터:
 tf.Tensor(
[[191.1  28.7  18.2]
 [215.4  23.6  57.6]
 [165.6  10.   17.6]
 [204.1  32.9  46. ]
 [187.9  17.2  17.9]], shape=(5, 3), dtype=float64)
Sales batch 데이터: tf.Tensor([17.3 17.1 12.6 19.  14.7], shape=(5,), dtype=float64)


### Keras

- 모델 클래스 객체 생성 : tf.keras.models.Sequential() / 괄호안에 layer 삽입
- tf.keras.layers.Dense(units, activation)
- units : 레이어 안의 노드 수
- activation : 적용할 활성화 함수 설정

- input layer는 입력 형태에 대한 정보를 필요로 함 (input_shape / input_dim 인자 설정)
- 제일 처음 레이어에는 input_dim을 지정해야 함, 뒤에는 상관없음

- 모델 학습 방식을 설정하기 위한 함수 : model.compile(optimizer, loss)
- optimizer : 모델 학습 최적화 방법 / loss : 손실 함수 설정


In [16]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_shape = (3,)),
    tf.keras.layers.Dense(1)
])

model.summary()

model.compile(loss = 'mean_squared_error', optimizer='adam')
history = model.fit(train_ds, epochs = 100, verbose=2)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 10)                40        
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
28/28 - 1s - loss: 11479.0205 - 1s/epoch - 47ms/step
Epoch 2/100
28/28 - 0s - loss: 6124.6172 - 58ms/epoch - 2ms/step
Epoch 3/100
28/28 - 0s - loss: 2981.0273 - 82ms/epoch - 3ms/step
Epoch 4/100
28/28 - 0s - loss: 1349.9916 - 64ms/epoch - 2ms/step
Epoch 5/100
28/28 - 0s - loss: 619.5861 - 94ms/epoch - 3ms/step
Epoch 6/100
28/28 - 0s - loss: 351.7637 - 95ms/epoch - 3ms/step
Epoch 7/100
28/28 - 0s - loss: 249.6100 - 99ms/epoch - 4ms/step
Ep

In [25]:
loss = model.evaluate(test_x, test_y) # 데스트 데이터 loss값 확인
for i in range(5) :
    print("%d 번째 테스트 데이터의 실제값: %f" % (i, test_y.iloc[i]))

2/2 [==============================] - 0s 3ms/step - loss: 4.5272
0 번째 테스트 데이터의 실제값: 13.400000
1 번째 테스트 데이터의 실제값: 13.200000
2 번째 테스트 데이터의 실제값: 15.000000
3 번째 테스트 데이터의 실제값: 9.900000
4 번째 테스트 데이터의 실제값: 26.200000


In [27]:
pred = model.predict(test_x) # 모델 예측 결과
for i in range(5) :
    print("%d 번째 테스트 데이터의 실제값: %f" % (i, pred[i][0]))

2/2 [==============================] - 0s 4ms/step
0 번째 테스트 데이터의 실제값: 14.415457
1 번째 테스트 데이터의 실제값: 11.899427
2 번째 테스트 데이터의 실제값: 17.523085
3 번째 테스트 데이터의 실제값: 6.627179
4 번째 테스트 데이터의 실제값: 26.404284


In [29]:
loss = model.evaluate(test_x,test_y,verbose = 0)
print("테스트 데이터의 loss : ",loss)

테스트 데이터의 loss :  4.527188777923584


### 신경망 모델로 분류

In [30]:
from sklearn.datasets import load_iris

x,y = load_iris(return_X_y=True)
df = pd.DataFrame(x, columns = ['꽃받침 길이','꽃받침 넓이','꽃잎 길이','꽃잎 넓이'])
df['클래스'] = y

x = df.drop(columns = ['클래스'])
y = df['클래스']
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state = 42)

train_ds = tf.data.Dataset.from_tensor_slices((train_x.values, train_y))
train_ds = train_ds.shuffle(len(train_x)).batch(batch_size = 5)


In [33]:
from tensorflow import keras

model = keras.models.Sequential([
    keras.layers.Dense(10, input_dim = 4),
    keras.layers.Dense(3, activation = 'softmax')
])

model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_ds, epochs = 100, verbose = 2)

Epoch 1/100
24/24 - 1s - loss: 1.5087 - accuracy: 0.2667 - 705ms/epoch - 29ms/step
Epoch 2/100
24/24 - 0s - loss: 1.3371 - accuracy: 0.1750 - 59ms/epoch - 2ms/step
Epoch 3/100
24/24 - 0s - loss: 1.2300 - accuracy: 0.2083 - 93ms/epoch - 4ms/step
Epoch 4/100
24/24 - 0s - loss: 1.1335 - accuracy: 0.2333 - 94ms/epoch - 4ms/step
Epoch 5/100
24/24 - 0s - loss: 1.0436 - accuracy: 0.3333 - 80ms/epoch - 3ms/step
Epoch 6/100
24/24 - 0s - loss: 0.9717 - accuracy: 0.5417 - 102ms/epoch - 4ms/step
Epoch 7/100
24/24 - 0s - loss: 0.9061 - accuracy: 0.6250 - 87ms/epoch - 4ms/step
Epoch 8/100
24/24 - 0s - loss: 0.8522 - accuracy: 0.6083 - 72ms/epoch - 3ms/step
Epoch 9/100
24/24 - 0s - loss: 0.8004 - accuracy: 0.6417 - 83ms/epoch - 3ms/step
Epoch 10/100
24/24 - 0s - loss: 0.7634 - accuracy: 0.6417 - 60ms/epoch - 2ms/step
Epoch 11/100
24/24 - 0s - loss: 0.7227 - accuracy: 0.6250 - 83ms/epoch - 3ms/step
Epoch 12/100
24/24 - 0s - loss: 0.6964 - accuracy: 0.6750 - 71ms/epoch - 3ms/step
Epoch 13/100
24/24 - 0

In [34]:
loss, acc = model.evaluate(test_x, test_y)
pred = model.predict(test_x)

print("테스트 데이터의 accuracy : ", acc)
for i in range(5):
    print("%d 번째 테스트 데이터의 실제값 : %d" % (i, test_y.iloc[i]))
    print("%d 번째 테스트 데이터 예측값: %d" % (i, np.argmax(pred[i])))
    
# np.argmax 함수는 함수 내에 array와 비슷한 형태(리스트 등 포함)의 input을 넣어주면 가장 큰 원소의 인덱스를 반환하는 형식

1/1 [==============================] - 0s 80ms/step
테스트 데이터의 accuracy :  0.9333333373069763
0 번째 테스트 데이터의 실제값 : 1
0 번째 테스트 데이터 예측값: 1
1 번째 테스트 데이터의 실제값 : 0
1 번째 테스트 데이터 예측값: 0
2 번째 테스트 데이터의 실제값 : 2
2 번째 테스트 데이터 예측값: 2
3 번째 테스트 데이터의 실제값 : 1
3 번째 테스트 데이터 예측값: 1
4 번째 테스트 데이터의 실제값 : 1
4 번째 테스트 데이터 예측값: 1
